In [96]:
import ee
import geemap
import os
import numpy as np

In [97]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)
brma2_region = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [98]:
Map.addLayer(brma2_region, {}, 'Manauas Region')

In [99]:
#Calculate NDRE for an image and set palette
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

In [100]:
#Create cloud mask
cloudBitMask = ee.Number(2).pow(10).int()
cirrusBitMask = ee.Number(2).pow(11).int()
def maskS2clouds(image):
  qa = image.select('QA60')
  # Both flags should be set to zero, indicating clear conditions.
  mask = (qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0)))
  return image.updateMask(mask)

In [101]:
#Load sentinel-2 data and clip to brma_2; set visual params
s2 = (ee.ImageCollection('COPERNICUS/S2')
     .map(lambda image : image.clip(brma2_region)))
visParams = {'bands' : ['B4', 'B3', 'B2'], 'max': 3000}

In [102]:
#Create image collections for each month
jan2016 = (s2.filterDate('2016-01-01', '2016-01-31')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
feb2016 = (s2.filterDate('2016-02-01', '2016-02-29')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
mar2016 = (s2.filterDate('2016-03-01', '2016-03-31')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
apr2016 = (s2.filterDate('2016-04-01', '2016-04-30')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
may2016 = (s2.filterDate('2016-05-01', '2016-05-31')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
jun2016 = (s2.filterDate('2016-06-01', '2016-06-30')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
jul2016 = (s2.filterDate('2016-07-01', '2016-07-31')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
aug2016 = (s2.filterDate('2016-08-01', '2016-08-31')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
sep2016 = (s2.filterDate('2016-09-01', '2016-09-30')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
oct2016 = (s2.filterDate('2016-10-01', '2016-10-31')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
nov2016 = (s2.filterDate('2016-11-01', '2016-11-30')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))
dec2016 = (s2.filterDate('2016-12-01', '2016-12-31')
           .map(maskS2clouds)
           .map(addNDRE).select('NDRE'))

In [103]:
#Create one median ndre image for each month
#PS I wish I could easily make a list and iterate but alas
#IRICI
jan16_med = ee.Image(jan2016.median())
feb16_med = ee.Image(feb2016.median())
mar16_med = ee.Image(mar2016.median())
apr16_med = ee.Image(apr2016.median())
may16_med = ee.Image(may2016.median())
jun16_med = ee.Image(jun2016.median())
jul16_med = ee.Image(jul2016.median())
aug16_med = ee.Image(aug2016.median())
sep16_med = ee.Image(sep2016.median())
oct16_med = ee.Image(oct2016.median())
nov16_med = ee.Image(nov2016.median())
dec16_med = ee.Image(dec2016.median())

In [104]:
#Test to cloud mask + ndre median
Map.addLayer(apr16_med, ndre_params, 'apr')
Map.addLayer(jul16_med, ndre_params, 'jul')
Map.addLayer(nov16_med, ndre_params, 'nov')

In [105]:
#Create one image collection with monthly median images + cloud masks
allMonths = ee.ImageCollection([jan16_med,
                                feb16_med,
                                mar16_med,
                                apr16_med,
                                may16_med,
                                jun16_med,
                                jul16_med,
                                aug16_med,
                                sep16_med,
                                oct16_med,
                                nov16_med,
                                dec16_med])

In [109]:
medianMonths = allMonths.median()
#Add new image collection to the map
Map.addLayer(medianMonths, ndre_params, 'Twelve Months NDRE')

In [112]:
#List sentinel2 metadata properties
properties = s2.first().propertyNames().getInfo()
print(properties)

['system:footprint', 'DATATAKE_IDENTIFIER', 'SPACECRAFT_NAME', 'FORMAT_CORRECTNESS_FLAG', 'IERS_BULLETIN_FILENAME', 'system:id', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8A', 'MEAN_SOLAR_AZIMUTH_ANGLE', 'SOLAR_IRRADIANCE_B12', 'system:version', 'SOLAR_IRRADIANCE_B10', 'SOLAR_IRRADIANCE_B11', 'GENERATION_TIME', 'SOLAR_IRRADIANCE_B8A', 'PRODUCT_URI', 'SENSOR_QUALITY_FLAG', 'CLOUD_COVERAGE_ASSESSMENT', 'system:time_end', 'system:time_start', 'DATASTRIP_ID', 'PROCESSING_BASELINE', 'SENSING_ORBIT_NUMBER', 'GEOMETRIC_QUALITY_FLAG', 'SENSING_ORBIT_DIRECTION', 'GRANULE_ID', 'REFLECTANCE_CONVERSION_CORRECTION', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B8', 'DATATAKE_TYPE', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B9', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B6', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B7', 'RADIOMETRIC_QUALITY_FLAG', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B4', 'MEAN_INCIDENCE_ZENITH_ANGLE_B1', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B5', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B2', 'MEAN_INCIDENCE_AZIMUTH_ANGLE_B3', 'MEAN_INCIDENCE_ZENITH_ANGLE_B5', 'MEAN_I

In [426]:
#Make sure you dont have 1 million images to download...
count = allMonths.size().getInfo()
print(count)

#Export all 12 images into a file
#This is quite slow, but it does work
out_dir = '/Users/morgansteckler/Desktop/All_Months/'
geemap.ee_export_image_collection(allMonths, out_dir, region = brma2_region, scale = 20, file_per_band=False)

Total number of images: 12

Exporting 1/12: 0.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/All_Months/0.tif


Exporting 2/12: 1.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/All_Months/1.tif


Exporting 3/12: 2.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/All_Months/2.tif


Exporting 4/12: 3.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/All_Months/3.tif


Exporting 5/12: 4.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/All_Months/4.tif


Exporting 6/12: 5.tif
Generating URL ...
Please wait ...


KeyboardInterrupt: 